In [2]:
import json
from pandas import *
import numpy as np
import os
import urllib
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2

In [3]:
data = []
with open("Indian_Number_plates.json") as file:
  for i in file:
    data.append(json.loads(i))


In [4]:
train_data, test_data = train_test_split(data, test_size = 0.1, random_state=42)

In [5]:
def download_data(image_f, data):
    for name, image in enumerate(tqdm(data)):
      img = urllib.request.urlopen(image['content'])
      img = Image.open(img).convert("RGB")
      img.save(f"/content/yolov5/numberplate/images/{image_f}/{name}.jpeg", "JPEG")


In [6]:
download_data("train", train_data)
download_data("validation", test_data)

100%|██████████| 24/24 [00:10<00:00,  2.24it/s]


In [7]:
train_image_path = "C:/Users/Raj/Numberplate detection/images/train"
test_image_path = "C:/Users/Raj/Numberplate detection/images/validation"

In [8]:
train_annotation = []
for i in train_data:
    temp = []
    for j in i['annotation']:
        temp.append(j)
    train_annotation.append(temp)

In [9]:
test_annotation = []
for i in test_data:
    temp = []
    for j in i['annotation']:
        temp.append(j)
    test_annotation.append(temp)

In [10]:
for i, element in enumerate(test_annotation):
    if len(element) > 1:
        print(i)

3


In [11]:
def prepare_file(data, name, datatype="train"):

    for element in data:
        temp = []
        p1, p2 = element['points']
        
        x1, y1 = p1['x'], p1['y']
        x2, y2 = p2['x'], p2['y']
        w = x2 - x1
        h = y2 - y1
        
        x_center = (x1 + x2)/2
        y_center = (y1 + y2)/2
        
        with open(f"/content/yolov5/numberplate/labels/{datatype}/{name}.txt", "a") as file:
            file.write(f"0 {x_center} {y_center} {w} {h} \n")
        

In [12]:
for i, annot in enumerate(tqdm(train_annotation)):
    prepare_file(annot, i, "train")


100%|██████████| 213/213 [00:00<00:00, 11296.11it/s]


In [13]:
for i, annot in enumerate(tqdm(test_annotation)):
    prepare_file(annot, i, "validation")

100%|██████████| 24/24 [00:00<00:00, 14130.17it/s]


In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 2048 (delta 47), reused 48 (delta 24), pack-reused 1965
Receiving objects: 100% (2048/2048), 5.21 MiB | 1.74 MiB/s, done.
Resolving deltas: 100% (1329/1329), done.


In [14]:
!pip3 install -r yolov5/requirements.txt

     |████████████████████████████████| 276kB 6.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=d7536305353519017460bc41a5d961de3063f99f0f66be779071aac00856e843
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
!python3 yolov5/train.py --img 640 --epochs 100 --data yolov5/numberplate.yaml --cfg yolov5/models/yolov5s.yaml --batch-size 8 --cache

Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=True, cfg='yolov5/models/yolov5s.yaml', data='yolov5/numberplate.yaml', device='', epochs=100, evolve=False, hyp='', img_size=[640, 640], local_rank=-1, multi_scale=False, name='', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=8, weights='', world_size=1)
Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
2020-08-04 04:20:32.847977: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.5, 'scale': 0.5, 'sh

In [19]:
!zip -r /content/file_run.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/exp0/ (stored 0%)
  adding: content/runs/exp0/weights/ (stored 0%)
  adding: content/runs/exp0/opt.yaml (deflated 44%)
  adding: content/runs/exp0/events.out.tfevents.1596514701.425dc8065a0f.357.0 (deflated 5%)
  adding: content/runs/exp0/hyp.yaml (deflated 42%)
  adding: content/runs/exp1/ (stored 0%)
  adding: content/runs/exp1/results.txt (deflated 78%)
  adding: content/runs/exp1/train_batch1.jpg (deflated 2%)
  adding: content/runs/exp1/weights/ (stored 0%)
  adding: content/runs/exp1/weights/last.pt (deflated 11%)
  adding: content/runs/exp1/weights/best.pt (deflated 11%)
  adding: content/runs/exp1/opt.yaml (deflated 44%)
  adding: content/runs/exp1/results.png (deflated 9%)
  adding: content/runs/exp1/train_batch2.jpg (deflated 3%)
  adding: content/runs/exp1/test_batch0_pred.jpg (deflated 2%)
  adding: content/runs/exp1/events.out.tfevents.1596514834.425dc8065a0f.388.0 (deflated 1%)
  adding: content/runs/exp1/hyp.yaml

In [21]:
!python3 yolov5/detect.py --source yolov5/numberplate/images/validation --weights runs/exp1/weights/best.pt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=640, iou_thres=0.5, output='inference/output', save_txt=False, source='yolov5/numberplate/images/validation', update=False, view_img=False, weights=['runs/exp1/weights/best.pt'])
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Fusing layers... Model Summary: 140 layers, 7.24652e+06 parameters, 6.61683e+06 gradients
image 1/24 /content/yolov5/numberplate/images/validation/0.jpeg: 512x640 Done. (0.011s)
image 2/24 /content/yolov5/numberplate/images/validation/1.jpeg: 576x640 2 numberplates, Done. (0.013s)
image 3/24 /content/yolov5/numberplate/images/validation/10.jpeg: 384x640 1 numberplates, Done. (0.013s)
image 4/24 /content/yolov5/numberplate/images/validation/11.jpeg: 512x640 1 numberplates, Done. (0.012s)
image 5/24 /content/yolov5/numberplate/images/validation/12.jpeg: 384x640 1 numberplates, Done. (0.011s)
image 6/24 /content/yolov5/numberpla